In [ ]:
import pandas as pd
from time import time

In [ ]:
start = time()

In [ ]:
# prepare weather

file_city_attributes = "./data/weather/city_attributes.csv"
file_humidity = "./data/weather/humidity.csv"
file_pressure = "./data/weather/pressure.csv"
file_temperature = "./data/weather/temperature.csv"
file_weather_description = "./data/weather/weather_description.csv"
file_wind_direction = "./data/weather/wind_direction.csv"
file_wind_speed = "./data/weather/wind_speed.csv"

In [ ]:
df_city_attributes = pd.read_csv(file_city_attributes)
df_city_attributes = df_city_attributes[df_city_attributes["City"]=="New York"]

In [ ]:
def extract_ny_data(source_file: str):
    df = pd.read_csv(source_file)
    df = df[["datetime","New York"]].reset_index()
    df = df.set_index(["datetime"])
    return df

In [ ]:
df_humidity = extract_ny_data(file_humidity)
df_pressure = extract_ny_data(file_pressure)
df_temperature = extract_ny_data(file_temperature)
df_weather_description = extract_ny_data(file_weather_description)
df_wind_direction = extract_ny_data(file_wind_direction)
df_wind_speed = extract_ny_data(file_wind_speed)

In [ ]:
df = df_humidity.join(df_pressure, on=["datetime"], lsuffix="_humidity")
df = df.join(df_temperature, on=["datetime"], lsuffix="_pressure")
df = df.join(df_weather_description, on=["datetime"], lsuffix="_temperature")
df = df.join(df_wind_direction, on=["datetime"], lsuffix="_weather_description")
df = df.join(df_wind_speed, on=["datetime"], lsuffix="_wind_direction")
index_columns = [col for col in df.columns if col.startswith("index")]
df = df.drop(index_columns, axis = 1)
df = df.rename(columns={
    "New York_humidity":"humidity",
    "New York_pressure":"pressure",
    "New York_temperature":"temperature",
    "New York_weather_description":"weather_description",
    "New York_wind_direction":"wind_direction",
    "New York":"wind_speed",
    })

In [ ]:
df.head()

In [ ]:
# find how many gaps in data

row_has_NaN = df.isnull().any(axis=1)
nan_count = len(df[row_has_NaN])
print("NaN:",nan_count)

In [ ]:
# conver strings to numeric values

def map_weather_to_number(weather):
    mapping={
        'few clouds': 1,
        'sky is clear': 2,
        'scattered clouds': 3,
        'broken clouds': 4,
        'overcast clouds': 5,
        'mist': 6,
        'drizzle': 7,
        'moderate rain': 8,
        'light intensity drizzle': 9,
        'light rain': 10,
        'fog': 11,
        'haze': 12,
        'heavy snow': 13,
        'heavy intensity drizzle': 14,
        'heavy intensity rain': 15,
        'light rain and snow': 16,
        'snow': 17,
        'light snow': 18,
        'freezing rain': 19,
        'proximity thunderstorm': 20,
        'thunderstorm': 21,
        'thunderstorm with rain': 22,
        'smoke': 23,
        'very heavy rain': 24,
        'thunderstorm with heavy rain': 25,
        'thunderstorm with light rain': 26,
        'squalls': 27,
        'dust': 28,
        'proximity thunderstorm with rain': 29,
        'thunderstorm with light drizzle': 30,
        'sand': 31,
        'shower rain': 32,
        'proximity thunderstorm with drizzle': 33,
        'light intensity shower rain': 34,
        'sand/dust whirls': 35,
        'heavy thunderstorm': 36
    }
    return mapping[weather] if weather in mapping else None

df["weather_description_code"] = df[["weather_description"]].applymap(map_weather_to_number)

In [ ]:
# fill gaps with interpolation

df = df.interpolate().fillna(method='bfill')

In [ ]:
df.reset_index().to_csv("./data/prepared_weather.csv", index=False)

In [ ]:
print(f"Done in {time()-start} sec")